In [1]:
import numpy as np
from netCDF4 import Dataset

In [2]:
import os
path = '/home/thirza/Cusize_python/PyRipser/python'
os.chdir(path)

from unionfind import UnionFind

In [3]:
ql_file = Dataset('/home/thirza/Cusize_python/subsubdomain_lasttimestep.nc','r',format='NETCDF4')
ncells = ql_file.variables['ncells'][:]

In [4]:
grid = Dataset('/home/thirza/Cusize_python/subsubgrid.nc','r',format='NETCDF4')
clon = grid.variables['clon'][:]
clat = grid.variables['clat'][:]
clon_vertices = grid.variables['clon_vertices'][:]
clat_vertices = grid.variables['clat_vertices'][:]
cell_area = grid.variables['cell_area'][:]
neighbor_cell_index = grid.variables['neighbor_cell_index'][:]

In [27]:
ql = ql_file.variables['qc'][0,:]
ql_proj = np.sum(ql,axis=0)
ql_binary = np.where(ql_proj>1E-8,1,0)

print ql_binary[347]
print neighbor_cell_index[:,108]
print neighbor_cell_index[:,109]

1
[108 115 357]
[182 112 117]


In [50]:
cloud_1 = np.array([1061, 1071, 1062, 1070, 1064, 1063, 3515])
print cloud_1
cloud_2 = np.array([9440, 9448, 9441, 9454, 9443, 9461, 9444])

[1061 1071 1062 1070 1064 1063 3515]


In [64]:
idx_clouds = np.array(np.where(ql_binary))[0]
nr_cloud_cells = len(idx_clouds)
test = UnionFind(nr_cloud_cells)

counter = 0

for j in range(0,nr_cloud_cells):
#for j in range(0,10):
    cell_1 = idx_clouds[j]
    
    for i in range(0,3):
        cell_2 = neighbor_cell_index[i,cell_1]-1  #python starts at 0, causing a difference of 1!
        if cell_2 != -1:
            if ql_binary[cell_2]: 
                idx_2 = np.where(idx_clouds==cell_2)
                test.link(j,idx_2)
                counter += 1

#print 'counter:',counter

for j in range(0, nr_cloud_cells):
    test.find(j)
    

indextoomuch = np.where(test.parent==563)
    
#print indextoomuch

#print idx_clouds[indextoomuch]

print neighbor_cell_index[:,9457]
print neighbor_cell_index[:,9461]

print clon_vertices[9457,:]
print clon_vertices[9461,:]



[  -1   -1 9462]
[9461 9455   -1]
[-0.99481574 -0.99485377 -0.99483454]
[-0.99483454 -0.99479651 -0.99481574]


In [47]:
cloud_label = 0
cloud_size_long = np.zeros(ncells)

parents = set(test.parent)
for p in parents:
    #print '%d --- %s' % (p, np.where(test.parent == p))
    idx_parent = np.where(test.parent == p)
    #print idx_parent
    cloud_cells = idx_clouds[idx_parent]
    #print cloud_cells
    if len(idx_parent[0])==8.:
        print 'cloud label:',cloud_label
        print 'p:',p
    cloud_size_long[cloud_label] = len(idx_parent[0])
    cloud_label += 1


cloud_size = np.trim_zeros(cloud_size_long)

#print cloud_size
nr_clouds = len(cloud_size)
print nr_clouds

sorted_sizes = np.sort(cloud_size)
print sorted_sizes


cloud label: 16
p: 563
33
[  1.   1.   1.   1.   1.   2.   3.   4.   4.   7.   8.   9.  10.  12.
  14.  17.  18.  24.  25.  27.  27.  29.  30.  32.  35.  36.  36.  50.
  57.  74.  87. 112. 139.]
